<a href="https://colab.research.google.com/github/geoaigroup/Aerial-SAM/blob/main/AerialSAM_GEOAI_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SAM Online Demo: Segment everything Mode

The Segment Anything Model (SAM) predicts object masks given prompts that indicate the desired object.

Please go tho this link:
https://segment-anything.com/demo

And use this image as input:
https://github.com/geoaigroup/Aerial-SAM/blob/main/483.png

## Environment Set-up

If running locally using jupyter, first install `segment_anything` in your environment using the [installation instructions](https://github.com/facebookresearch/segment-anything#installation) in the repository. If running from Google Colab, set `using_colab=True` below and run the cell. In Colab, be sure to select 'GPU' under 'Edit'->'Notebook Settings'->'Hardware accelerator'.

In [ ]:
#Samples used in this demo are from the WHU Building Dataset: https://paperswithcode.com/dataset/whu-building-dataset
!wget https://github.com/geoaigroup/Aerial-SAM/raw/main/resources/data.zip
!wget https://github.com/geoaigroup/Aerial-SAM/raw/main/resources/pred_shapefile.zip
!unzip data.zip
!unzip pred_shapefile

In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
    !pip install geopandas
    !pip install rasterio
    !git clone https://github.com/geoaigroup/buildingsSAM.git


In [ ]:
#Necessary imports and helper functions for displaying points, boxes, and masks.
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import geopandas as gpd
import os
import json
import glob
from tqdm import tqdm
import shapely.geometry as sg
from shapely import affinity
from shapely.geometry import Point, Polygon
import random
from PIL import Image, ImageDraw
import rasterio
from rasterio.features import geometry_mask
#from metrics import DiceScore,IoUScore
import pandas as pd
import gc
import shutil
import fiona
import json

import utils
from evaluate import cal_scores,matching_algorithm
from pred_SAM import SAM


In [ ]:

def cal_score(gt_tile, pred_tile):
    matcher = matching_algorithm(gt_tile, pred_tile)
    iou_list, f1_scores, tp_pred_indices, tp_gt_indices, fp_indices, fn_indices, mscores, precision, recall = matcher.matching()
    tp_iou_list, avg_tp_iou = matcher.tp_iou(tp_pred_indices, tp_gt_indices)
    score = {}
    scores_b = []
    score['iou_list'] = iou_list
    score['f1_scores'] = f1_scores
    score['tp_iou_list'] = tp_iou_list
    score['fp_indices'] = fp_indices
    score['fn_indices'] = fn_indices
    score['Mean_iou'] = np.mean(iou_list, dtype=float)
    score['Mean_f1'] = np.mean(f1_scores, dtype=float)
    score['avg_tp_iou'] = float(avg_tp_iou) if avg_tp_iou != None else 0.0
    score['precision'] = precision
    score['recall'] = recall

    for s in mscores:
        scores_b.append(s)
    scores_b.append(score)

    gtmask=np.zeros((512,512))
    predmask=np.zeros((512,512))
    for g in gt_tile:
        gtmask=g+gtmask
    for p in pred_tile:
        predmask=p+predmask
    fig,ax = plt.subplots(1,2,figsize = (10,10))
    ax = ax.ravel()
    ax[0].imshow(gtmask)
    ax[0].set_title("GT")
    ax[1].imshow(predmask)
    ax[1].set_title("MultiClassUnet CNN")
    plt.show()

    return scores_b

def Calculate_CNN_Results():
    ff = gpd.read_file(pred)
    score_list = []

    ids = [f for f in os.listdir(orig_shp)]

    for name in tqdm(ids):
        print(name)
        if glob.glob(score_dir + "/" + name + "_score.json" ):
            print("Found")
            continue
        if name in os.listdir(orig_shp):
            try:
                gt = gpd.read_file(orig_shp + "/" + name)
                if len(gt["geometry"]) == 0:
                    continue
            except Exception as e:
                print(e)
                continue
        else:
            continue
        predic = ff.loc[ff["ImageId"] == name]
        n=name.split('.')[0]
        if len(predic["geometry"]) == 0:
            continue


        gc.collect()

        gt_tile = []
        pred_tile=[]

        gt_tile=utils.convert_polygon_to_mask_batch(gt['geometry'])
        pred_tile=utils.convert_polygon_to_mask_batch(predic["geometry"])

        scores_res=cal_score(gt_tile, pred_tile)
        os.makedirs(score_dir, exist_ok=True)

        with open(score_dir + f'/{name}_score.json', 'w') as f1:
            json.dump(scores_res, f1)

    scores=cal_scores(output_dir,score_dir)
    scores.macro_score()

In [ ]:
def main(CNN="",prompt_type="",sam=None):
    score_list = []
    scores=cal_scores(output_dir,score_dir)
    # ff = gpd.read_file(pred)
    ids = [f for f in os.listdir(orig_shp)]
    for name in tqdm(ids):
        print(name)
        print("Checking")
        flag=0
        if glob.glob(output_dir + "/" + name + "/" + name + ".shp" ) or glob.glob(output_dir + "/" + name + "/" + name + ".png" ):
            print("Found")
            continue

        tile_boxes = []
        try:
            image = cv2.imread(images + "/" + name+'.png')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(e)
            print(name)

        if name in os.listdir(orig_shp):
                gt = gpd.read_file(orig_shp + "/" + name)
                if len(gt["geometry"]) == 0:
                    continue
        else:
            continue
        if CNN=="multiclassUnet":

            predic = ff.loc[ff["ImageId"] == name]

        elif CNN=="DCNN":
            predic = gpd.read_file(pred+"/"+name)

        geo = predic["geometry"]

        if len(geo) == 0:
            continue

        input_point=None
        input_label=None
        input_boxes=None
         
        match prompt_type:
            case "single point":
                input_point,input_label=utils.create_list_points(geo,name)
            case "single + negative points":
                input_point,input_label=utils.create_list_points(geo,name,flag="negative")
                print(input_point)
                print(input_label)
            #Skeleton
            case "skeleton":
                input_point=[]
                input_label=[]
                with open(skeleton_points, 'r') as json_file:
                    data = json.load(json_file)
                matching_items = []
                for item in data:
                    if item['id'] == name:
                        matching_items.append(item)

                input_point=torch.Tensor(matching_items[0]['input_points']).cuda()
                input_label=torch.Tensor(matching_items[0]['input_labels']).cuda().long()

            case "multiple points":
                input_point,input_label=utils.generate_random_points_polygon(geo)
            
            case "multiple points + single point":
                input_point,input_label=utils.generate_random_points_polygon(geo,flag="rep")

            case "multiple points + negative points":
                input_point,input_label=utils.generate_random_points_polygon(geo,flag="negative")
           
            #creating boxes
            case "box":
                input_boxes=[]
                flag=1
                tile_boxes=utils.create_boxes(geo)
                input_boxes=torch.tensor(tile_boxes).cuda()
            
            case "box + single point":
                input_boxes=[]   
                tile_boxes=utils.create_boxes(geo)
                input_boxes=torch.tensor(tile_boxes).cuda()
                input_point,input_label=utils.create_list_points(geo,name)

            case "box + multiple points":
                input_boxes=[]
                tile_boxes=utils.create_boxes(geo)
                input_boxes=torch.tensor(tile_boxes).cuda()
                input_point,input_label=utils.generate_random_points_polygon(geo)

            case _:
                print("no or wrong prompt entered")

        x = torch.from_numpy(image.transpose(2, 0, 1)).float().cuda()
        pred_mask=sam.predictSAM(x=x,image=image,input_point=input_point,input_label=input_label,input_boxes=input_boxes,flag=flag)
        os.makedirs(score_dir, exist_ok=True)
        os.makedirs(output_dir + "/" + f"{name}", exist_ok=True)

        scores.micro_match_iou(pred_mask,name,gt,score_list,image,input_point,input_label,tile_boxes,geo=geo)
    scores.macro_score()


In [ ]:
# Paths
checkpoint="data/checkpoint/sam_vit_h_4b8939.pth"
images = "data/images"
orig_shp="data/orig_shp"
skeleton_points="data/points.json"
pred = "data/pred_shapefile"
output_dir = "data/MulticlassUdfnet_0-0=s_output"
score_dir = "data/MulticlassUndfet_00-oi0nts_scores"

In [ ]:
#get Multiclass Unet initial results
# Calculate_CNN_Results()

#loading SAM Model
sam=SAM(checkpoint)

#load Multiclass Unet CNN prediction file
ff = gpd.read_file(pred)

#Run SAM prediction with box prompt
main(CNN="multiclassUnet",prompt_type="box",sam=sam)


In [ ]:
#for D-linkNet model
pred = "data/DCNN_pred_shapefile"
sam=SAM(checkpoint)
main(CNN="DCNN",prompt_type="box",sam=sam)

LangSAM : https://github.com/luca-medeiros/lang-segment-anything

GroundingDINO: https://github.com/IDEA-Research/GroundingDINO

Segment Anything : https://github.com/facebookresearch/segment-anything

**Language Segment-Anything (LangSAM)** is an open-source project that combines the power of instance segmentation and text prompts to generate masks for specific objects in images. Built on the recently released Meta model, segment-anything, and the GroundingDINO detection model, it's an easy-to-use and effective tool for object detection and image segmentation.

This section was prepared by Hasan Moughnieh.

In [ ]:
!wget https://github.com/geoaigroup/Aerial-SAM/raw/main/resources/LangSAM_samples.zip
!wget https://github.com/geoaigroup/Aerial-SAM/raw/main/resources/LangSAM_scripts.zip
!unzip LangSAM_samples.zip
!unzip LangSAM_scripts.zip

In [ ]:
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow


Grounding DINO:

Image + Text --> 900 bounding boxes(by default) each with similarity score with respect to the input words

900 boxes --> filtered according to the box_threshold (similarity score > box_threshold)

extract the words whose similarities are higher than the text_threshold as predicted labels.

The optimal threshold can vary depending on the quality and nature of your images, as well as the specificity of your text prompts.


In [ ]:
from PIL import Image
from LangSAM import LangSAM
from demo_functions import load_ground_truth_masks , display_images_with_masks

index = '2_36'
#replace 2_37 or 2_102 for more good samples
# replace with 0 or 2_87 for some bad results
# or upload an image of your own
image = f'/content/{index}_img.png'
ground_truth_mask = f'/content/{index}_gt.png'

model = LangSAM()

image_pil = Image.open(image).convert("RGB")
text_prompt = "house" #here you can change your textual input(query)
masks, boxes, phrases, logits = model.predict(image_pil, text_prompt)
ground_truth_masks = load_ground_truth_masks(ground_truth_mask)

#This function displays the original image , predicted masks , and accuracy compared to ground truth
display_images_with_masks(image_pil, masks , boxes, ground_truth_mask)

Evaluation

In [ ]:
from demo_functions import MaskMatchingAlgorithm

matcher = MaskMatchingAlgorithm(ground_truth_mask, masks)
output = matcher.matching()
iou_list = output[0]
tp_pred_indices = output[1]
tp_gt_indices = output[2]
fp_indices = output[3]
fn_indices = output[4]
tp_iou_list, avg_tp_iou = matcher.tp_iou(tp_pred_indices, tp_gt_indices)
average_iou, avg_tp_iou , precision , recall , f1_score, tp_f1 = matcher.display_results(iou_list ,tp_pred_indices,
                                                                                                tp_gt_indices,
                                                                                                fp_indices,
                                                                                                fn_indices ,
                                                                                                tp_iou_list,
                                                                                                avg_tp_iou)
print(" F1 score" , tp_f1)